In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
#end_time = pd.to_datetime('2014-11-01')

infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_short_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_short_jobs_real_pow"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))

There are 35284 long jobs in the Andrea's interval


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Calculate the mean core consumption

In [7]:
# clean user names
long_jobs['user_short'] = long_jobs.apply(lambda row: row['user'].split('@')[0], axis=1)

In [8]:
long_jobs['user'].unique().shape[0]

232

In [9]:
long_jobs['user_short'].unique().shape[0]

157

In [16]:
#long_jobs_quality  = long_jobs[long_jobs['real_pow_quality'] > 0.5]

In [27]:
grouped_user_data = long_jobs.groupby('user_short')['job_id_string'].nunique().reset_index(name='counts')
print("{} distinct users".format(grouped_user_data.shape[0]))

grouped_user_data['mean_2_1'] = 0.0
grouped_user_data['mean_2_2'] = 0.0
grouped_user_data['mean_3_1'] = 0.0
grouped_user_data['mean_overAll'] = 0.0

157 distinct users


In [28]:
grouped_user_data['counts'].sum()

35284

In [29]:
i = 0
for user_index, user_row in grouped_user_data.iterrows():
    user_jobs = long_jobs[long_jobs['user_short'] == user_row['user_short']]
    user_jobs_2_1 = user_jobs[user_jobs['node_req'] == user_jobs['n_2_1']]
    user_jobs_2_2 = user_jobs[user_jobs['node_req'] == user_jobs['n_2_2']]
    user_jobs_3_1 = user_jobs[user_jobs['node_req'] == user_jobs['n_3_1']]
    
    if(user_jobs_2_1['cpu_req'].sum() != 0):
        mean_core_2_1 = user_jobs_2_1['real_pow'].sum() / user_jobs_2_1['cpu_req'].sum()
        grouped_user_data.at[user_index, 'mean_2_1'] = mean_core_2_1
    if(user_jobs_2_2['cpu_req'].sum() != 0):
        mean_core_2_2 = user_jobs_2_2['real_pow'].sum() / user_jobs_2_2['cpu_req'].sum()
        grouped_user_data.at[user_index, 'mean_2_2'] = mean_core_2_2
    if(user_jobs_3_1['cpu_req'].sum() != 0): 
        mean_core_3_1 = user_jobs_3_1['real_pow'].sum() / user_jobs_3_1['cpu_req'].sum()
        grouped_user_data.at[user_index, 'mean_3_1'] = mean_core_3_1
    
    if(user_jobs['cpu_req'].sum() != 0):
        mean_core_overAll = user_jobs['real_pow'].sum() / user_jobs['cpu_req'].sum()
        grouped_user_data.at[user_index, 'mean_overAll'] = mean_core_overAll
    #print("{}, {}, {}, {}".format(user_row['user'], mean_core_2_1, mean_core_2_2, mean_core_3_1))
    
    if(i % 50 == 0):
        print(i)
    i += 1

0
50
100
150


In [30]:
grouped_user_data
# Problemi: utenti che non hanno eseguito job su singoli nodi

,user_short,counts,mean_2_1,mean_2_2,mean_3_1,mean_overAll
0,a07smr01,1003,0.000000,0.000000,12.673113,12.673113
1,a08tra59,61,13.682585,9.169212,6.584470,10.363496
2,a08tra60,19,5.471959,14.731025,3.950499,5.130931
3,a08tra61,74,7.590298,5.349340,18.053879,7.850219
4,a08tra62,1,0.000000,0.000000,3.425607,3.425607
5,a08tra63,20,9.246161,11.665610,14.803861,11.711186
6,a08tra64,25,34.234771,7.124345,13.778326,24.967776
7,aalbinat,20,0.000000,28.372177,31.301122,30.850515
8,aambrose,160,12.940221,16.370231,30.044172,17.818265
9,abiancar,12,16.910036,0.000000,18.561361,18.364774


In [31]:
outfile_grouped_user_data =  datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_consumption_users"
grouped_user_data.to_csv(outfile_grouped_user_data + ".csv")

## Predict

In [32]:
infile_mean_core_consumptions = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_consumption_users"
mean_core_consumptions = pd.read_csv(infile_mean_core_consumptions + ".csv", index_col=0)

long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))
# clean user names
long_jobs['user_short'] = long_jobs.apply(lambda row: row['user'].split('@')[0], axis=1)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 35284 long jobs in the Andrea's interval


In [33]:
mean_core_consumptions.sort_values('counts', ascending=False)

,user_short,counts,mean_2_1,mean_2_2,mean_3_1,mean_overAll
27,asaetti0,11401,0.000000,47.849953,35.696040,35.947625
149,szia0000,6564,0.000000,0.000000,20.570443,20.570443
150,tbusatta,1831,26.050281,32.574553,48.129747,37.562039
64,fpetracc,1624,12.873880,13.458109,27.756758,17.160570
50,emarcucc,1340,18.599510,19.205553,33.017027,28.661398
0,a07smr01,1003,0.000000,0.000000,12.673113,12.673113
45,dmelazzi,655,33.887636,31.257730,58.511135,44.104548
87,jfranz00,574,12.080244,15.463817,30.479593,21.223651
62,fnegreir,564,16.115350,19.508816,31.859560,26.399102
112,mmapelli,509,0.000000,0.000000,31.575779,31.575779


In [34]:
long_jobs['pred_user'] = 0.0

In [35]:
i = 0
for job_index, job_row in long_jobs.iterrows():
    job_user = job_row['user_short']
    job_n_cores_2_1 = job_row['n_cores_2_1']
    job_n_cores_2_2 = job_row['n_cores_2_2']
    job_n_cores_3_1 = job_row['n_cores_3_1']
    
    
    mean_core_consumptions_2_1 = mean_core_consumptions[mean_core_consumptions['user_short'] == job_row['user_short']].iloc[0][2]
    if(mean_core_consumptions_2_1 == 0.0):
        mean_core_consumptions_2_1 = mean_core_consumptions[mean_core_consumptions['user_short'] == job_row['user_short']].iloc[0][5]
    mean_core_consumptions_2_2 = mean_core_consumptions[mean_core_consumptions['user_short'] == job_row['user_short']].iloc[0][3]
    if(mean_core_consumptions_2_2 == 0.0):
        mean_core_consumptions_2_2 = mean_core_consumptions[mean_core_consumptions['user_short'] == job_row['user_short']].iloc[0][5]
    mean_core_consumptions_3_1 = mean_core_consumptions[mean_core_consumptions['user_short'] == job_row['user_short']].iloc[0][4]
    if(mean_core_consumptions_3_1 == 0.0):
        mean_core_consumptions_3_1 = mean_core_consumptions[mean_core_consumptions['user_short'] == job_row['user_short']].iloc[0][5]
    
    job_predicted_consumption = job_n_cores_2_1 * mean_core_consumptions_2_1 + job_n_cores_2_2 * mean_core_consumptions_2_2 + job_n_cores_3_1 * mean_core_consumptions_3_1
    
    long_jobs.at[job_index, 'pred_user'] = job_predicted_consumption
    
    if (i % 10000 == 0):
        print(i)
    i += 1

0
10000
20000
30000


In [36]:
# write to long_jobs
long_jobs.to_csv(infile_long_jobs + ".csv")

# Validate

In [3]:
from sklearn.metrics import r2_score
from scipy import stats

from sklearn.metrics import mean_squared_error

In [5]:
good_quality_jobs = long_jobs[long_jobs['real_pow_quality'] != 0]
#print(math.sqrt(np.var(good_quality_jobs['real_pow'])))
#good_quality_jobs = long_jobs

mean_real_pow = long_jobs['real_pow'].mean()
print("the mean is {}".format(mean_real_pow))


good_quality_jobs['numerator'] = (good_quality_jobs['real_pow'] - good_quality_jobs['pred_user'])**2
good_quality_jobs['denumerator'] = (good_quality_jobs['real_pow'] - mean_real_pow)**2
N = good_quality_jobs['numerator'].shape[0]

r_2 = 1 - (good_quality_jobs['numerator'].sum() / good_quality_jobs['denumerator'].sum())
print("r_2 is {}".format(r_2))

rmse = math.sqrt(good_quality_jobs['numerator'].sum() / N)
print("RMSE is {}".format(rmse))
print("NRMSE is {}".format(rmse/mean_real_pow))

the mean is 110.35204538702968
r_2 is 0.7560123903952256
RMSE is 133.21978340424243
NRMSE is 1.2072253208992223


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [5]:
#NRMSE
good_quality_jobs['variance'] = (good_quality_jobs['real_pow'] - good_quality_jobs['pred_user'])**2
math.sqrt(good_quality_jobs['variance'].sum() / good_quality_jobs.shape[0]) / good_quality_jobs['real_pow'].mean()

1.1858111396650617

In [9]:
# R^2
good_quality_jobs['variance_2'] = (good_quality_jobs['real_pow'] - good_quality_jobs['real_pow'].mean())**2
1-(good_quality_jobs['variance'].sum() / good_quality_jobs['variance_2'].sum())   

0.751802699842797

In [73]:
# consider only the overAll
i = 0
for job_index, job_row in long_jobs.iterrows():
    job_user = job_row['user_short']
    job_n_2_1 = job_row['n_2_1']
    job_n_2_2 = job_row['n_2_2']
    job_n_3_1 = job_row['n_3_1']
    

    mean_core_cons = mean_core_consumptions[mean_core_consumptions['user_short'] == job_user].iloc[0][5]
    
    job_predicted_consumption = job_n_2_1 * mean_core_cons + job_n_2_2 * mean_core_cons + job_n_3_1 * mean_core_cons
    
    long_jobs.at[job_index, 'pred_user'] = job_predicted_consumption
    
    if (i % 10000 == 0):
        print(i)
    i += 1

0
10000
20000
30000


In [75]:
slope, intercept, r_value, p_value, std_err = stats.linregress(long_jobs['real_pow'], long_jobs['pred_user'])
print("R-squared: {}".format(r_value**2))
from sklearn.metrics import mean_squared_error
print("mean_quared_error: {}".format(mean_squared_error(long_jobs['real_pow'], long_jobs['pred_user'])))

R-squared: 0.24692075463308644
mean_quared_error: 71464.7445047916


## Debug

In [ ]:
    user_jobs_2_1['predicted'] = mean_core_2_1 * user_jobs_2_1['cpu_req']
    user_jobs_2_2['predicted'] = mean_core_2_2 * user_jobs_2_2['cpu_req']
    user_jobs_3_1['predicted'] = mean_core_3_1 * user_jobs_3_1['cpu_req']
    
    if(user_jobs_2_1.shape[0] != 0):
        slope, intercept, r_value, p_value, std_err = stats.linregress(user_jobs_2_1['real_pow'], user_jobs_2_1['predicted'])
        grouped_user_data.loc[node_index, 'R-squared_2_1'] = r_value**2
        grouped_user_data.loc[node_index, 'count_2_1'] = user_jobs_2_1.shape[0]
    if(user_jobs_2_2.shape[0] != 0):
        slope, intercept, r_value, p_value, std_err = stats.linregress(user_jobs_2_2['real_pow'], user_jobs_2_2['predicted'])
        grouped_user_data.loc[node_index, 'R-squared_2_2'] = r_value**2
        grouped_user_data.loc[node_index, 'count_2_2'] = user_jobs_2_2.shape[0]
    if(user_jobs_3_1.shape[0] != 0):
        slope, intercept, r_value, p_value, std_err = stats.linregress(user_jobs_3_1['real_pow'], user_jobs_3_1['predicted'])
        grouped_user_data.loc[node_index, 'R-squared_3_1'] = r_value**2
        grouped_user_data.loc[node_index, 'count_3_1'] = user_jobs_3_1.shape[0]

In [19]:
grouped_user_data['R-squared_2_1'] = grouped_user_data['R-squared_2_1'].round(6)
grouped_user_data['R-squared_2_2'] = grouped_user_data['R-squared_2_2'].round(6)
grouped_user_data['R-squared_3_1'] = grouped_user_data['R-squared_3_1'].round(6)

In [20]:
grouped_user_data[grouped_user_data['count_3_1'] != 1].sort_values('counts', ascending=False)

,user,counts,count_2_1,R-squared_2_1,count_2_2,R-squared_2_2,count_3_1,R-squared_3_1
129,asaetti0@node129.eurora.cineca.it,9216,0,0.000000,337,0.000000,8879,0.000000
220,fpetracc@node129.eurora.cineca.it,2684,1595,0.000000,683,0.000000,406,0.000000
218,fnicolus@node129.eurora.cineca.it,1763,10,0.000000,2,0.000000,1751,0.000005
217,fnegreir@node129.eurora.cineca.it,1735,413,0.934622,171,0.639407,1148,0.874153
319,mdistefa@node129.eurora.cineca.it,1689,506,0.994646,129,0.585681,1054,0.993419
488,tbusatta@node129.eurora.cineca.it,1674,646,0.000000,204,0.000000,824,0.000000
320,mdotti00@node129.eurora.cineca.it,1575,979,0.731976,327,0.099297,269,0.005865
348,mpalummo@node129.eurora.cineca.it,1349,483,0.830202,273,0.706958,479,0.473446
191,emarcucc@node129.eurora.cineca.it,1286,378,0.000000,28,0.000000,880,0.000000
234,gbertain@node129.eurora.cineca.it,1262,580,0.848754,131,0.192991,520,0.547021
